In [1]:
!pip install -U imgaug==0.2.5
!pip install -U scipy==1.2.0

Requirement already up-to-date: imgaug==0.2.5 in /usr/local/lib/python3.6/dist-packages (0.2.5)
Requirement already up-to-date: scipy==1.2.0 in /usr/local/lib/python3.6/dist-packages (1.2.0)


In [2]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.applications import vgg19
from keras import backend as K

Using TensorFlow backend.


In [0]:
import numpy as np

In [0]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

In [0]:
target_image_path = './drive/My Drive/ML/keras_book/Generate/StyleChange/origine.jpg'
style_reference_image_path = './drive/My Drive/ML/keras_book/Generate/StyleChange/reference2.jpg'

In [0]:
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [0]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [0]:
def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x= np.clip(x, 0, 255).astype('uint8')
    return x

In [10]:
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

W0624 13:24:58.839168 140674872330112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [0]:
input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

In [12]:
model = vgg19.VGG19(
    input_tensor = input_tensor,
    weights = 'imagenet',
    include_top = False
)

W0624 13:24:58.863802 140674872330112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0624 13:24:58.870327 140674872330112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0624 13:24:58.898812 140674872330112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0624 13:24:59.294869 140674872330112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0624 13:24:59.295957 140674872330112 d

In [0]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [0]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [0]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a+b, 1.25))

In [0]:
output_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1'
]

In [0]:
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025
loss = K.variable(0.)

In [19]:
layer_features = output_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

W0624 13:25:00.328723 140674872330112 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


In [0]:
for layer_name in style_layers:
    layer_features = output_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
    
loss += total_variation_weight * total_variation_loss(combination_image)

In [21]:
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

W0624 13:25:00.620480 140674872330112 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
      
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [0]:
evaluator = Evaluator()

In [0]:
result_prefix = 'style_transfer_result'
iterations = 20

In [0]:
x = preprocess_image(target_image_path)
x = x.flatten()

In [26]:
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(
        evaluator.loss,
        x,
        fprime = evaluator.grads,
        maxfun = 20
    )
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' %i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 5074631700.0
Image saved as style_transfer_result_at_iteration_0.png
Iteration 0 completed in 12s
Start of iteration 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Current loss value: 1946312300.0
Image saved as style_transfer_result_at_iteration_1.png
Iteration 1 completed in 7s
Start of iteration 2
Current loss value: 1297077500.0
Image saved as style_transfer_result_at_iteration_2.png
Iteration 2 completed in 7s
Start of iteration 3
Current loss value: 1016613950.0
Image saved as style_transfer_result_at_iteration_3.png
Iteration 3 completed in 7s
Start of iteration 4
Current loss value: 865360600.0
Image saved as style_transfer_result_at_iteration_4.png
Iteration 4 completed in 7s
Start of iteration 5
Current loss value: 768883500.0
Image saved as style_transfer_result_at_iteration_5.png
Iteration 5 completed in 7s
Start of iteration 6
Current loss value: 695353800.0
Image saved as style_transfer_result_at_iteration_6.png
Iteration 6 completed in 7s
Start of iteration 7
Current loss value: 647296200.0
Image saved as style_transfer_result_at_iteration_7.png
Iteration 7 completed in 7s
Start of iteration 8
Current loss value: 609544000.0
Image 